In [1]:
# !pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip3 install gspread oauth2client
# !pip3 install requests
# !pip3 install langchain
# !pip3 install openai
# !pip3 install gspread oauth2client google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
# !pip3 install python-dotenv

# Set up environment

In [1]:
# for the predictions
import os
import sys
import sqlite3
import pathlib
import pandas as pd
from dotenv import load_dotenv

# my funcitons
sys.path.append("use-predictions") 
import sending_functions as sf

# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"
secrets_path = project_root / "secrets.env"
json_path = project_root / "service-account-token.json"

# lead secrets
load_dotenv(dotenv_path=secrets_path)

True

In [3]:
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open('prediction_table.sql', 'r') as file:
    query = file.read()

# Execute the query and fetch the results into a data frame
predictions = pd.read_sql_query(query, con)

# Disconnect from the SQLite database
con.close()

predictions

,game_id,home_team_result,team_home,position_home,team_head_to_head_odds_home,team_away,position_away,team_head_to_head_odds_away,home_team_win_prob,home_team_lose_prob,round_id,competition_year,round_name
0,20231111810,Win,Cronulla-Sutherland Sharks,4,1.11,St. George Illawarra Dragons,17,6.70,0.708443,0.291557,18,2023,Round 18
1,20231111820,Win,New Zealand Warriors,5,1.70,South Sydney Rabbitohs,8,2.16,0.553274,0.446726,18,2023,Round 18
2,20231111830,Win,Melbourne Storm,3,1.87,Penrith Panthers,1,1.97,0.552467,0.447533,18,2023,Round 18
3,20231111840,Win,Canberra Raiders,6,1.63,Gold Coast Titans,9,2.28,0.652813,0.347187,18,2023,Round 18
4,20231111850,Win,North Queensland Cowboys,10,1.15,Wests Tigers,16,5.50,0.710377,0.289623,18,2023,Round 18
5,20231111860,Win,Brisbane Broncos,2,1.34,Dolphins,11,3.27,0.697508,0.302492,18,2023,Round 18
6,20231111870,Win,Canterbury-Bankstown Bulldogs,15,2.12,Newcastle Knights,14,1.72,0.553493,0.446507,18,2023,Round 18
7,20231111880,Win,Manly-Warringah Sea Eagles,13,1.88,Sydney Roosters,12,1.96,0.660299,0.339701,18,2023,Round 18


# Get Tipper Picks

In [4]:
tipper_picks = sf.get_tipper_picks(predictions)
tipper_picks

,team,price,price_min
6,Canterbury-Bankstown Bulldogs,2.12,1.806708
7,Manly-Warringah Sea Eagles,1.88,1.514465


# Save predicitons

In [6]:
sf.upload_df_to_drive(
    predictions, 
    json_path, 
    os.getenv('FOLDER_ID'), 
    "predictions.csv"
)

File ID: 1N0iKpf3SZuxK6UwbKIuFMpiURfNwti6m


# Reg R-ai-gan

In [9]:
reg_regan = sf.generate_reg_regan_email(
    predictions, 
    tipper_picks, 
    os.getenv('OPENAI_KEY'), 
    os.getenv('FOLDER_URL')
)

print(reg_regan)

Subject: NRL Round 18 Predictions and Tips from Reg Regan

G'day mates,

Hope this email finds you all in good spirits and ready for some NRL action! Reg Regan here, your go-to footy expert with some interesting predictions for Round 18. Our trusty machine learning pipeline, the Footy Tipper, has shared its insights, so let's dive right in!

Here's a rundown of the upcoming matches:

1. Cronulla-Sutherland Sharks vs. St. George Illawarra Dragons:
   - The Sharks are sitting comfortably at the 4th position while the Dragons are struggling down at 17th. It's safe to say that the Sharks have this one in the bag.
   - Home Team Result: Win
   - Home Team: Cronulla-Sutherland Sharks
   - Home Team Position: 4
   - Home Team Head to Head Price: 1.11
   - Away Team: St. George Illawarra Dragons
   - Away Team Position: 17
   - Away Team Head to Head Price: 6.7

2. New Zealand Warriors vs. South Sydney Rabbitohs:
   - The Warriors, currently at 5th, will clash with the Rabbitohs, positioned 8t

# Send emails

In [6]:
sf.send_emails(
    "footy-tipper-email-list", 
    f"Footy Tipper Predictions for {predictions['round_name'].unique()[0]}", 
    reg_regan, 
    os.getenv('MY_EMAIL'), 
    os.getenv('EMAIL_PASSWORD'), 
    json_path
)

# # Use your downloaded credentials file
# scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
#          "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# # creds = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
# client = gspread.authorize(creds)

# # Open the test sheet and get the data
# sheet = client.open("footy-tipper-email-list").sheet1  # use your actual sheet name
# email_data = sheet.get_all_records()  # gets all the data inside your Google Sheet

# your_mates_emails = [row['Email'] for row in email_data]  # replace 'Email' with your actual column name

# # Setup the email
# msg = MIMEMultipart()
# msg['From'] = os.getenv('MY_EMAIL')
# msg['To'] = ', '.join(your_mates_emails)
# msg['Subject'] = f"Footy Tipper Predictions for {predictions['round_name'].unique()[0]}"

# # Add your message
# message = reg_regan  # or whatever variable has your final message
# msg.attach(MIMEText(message, 'plain'))

# # Setup the SMTP server
# server = smtplib.SMTP('smtp.gmail.com', 587)
# server.starttls()

# # Add your credentials
# server.login(os.getenv('MY_EMAIL'), os.getenv('EMAIL_PASSWORD'))

# # Send the email
# text = msg.as_string()
# server.sendmail(os.getenv('MY_EMAIL'), your_mates_emails, text)

# # Close the connection
# server.quit()

(221,
 b'2.0.0 closing connection p38-20020a634f66000000b005535ddd8dcfsm5431199pgl.89 - gsmtp')